In [3]:
import open3d as o3d
import numpy as np
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
pcd = o3d.io.read_point_cloud('flat.ply')
assert isinstance(pcd, o3d.geometry.PointCloud)
print(pcd)

PointCloud with 4297125 points.


In [5]:
o3d.visualization.draw_geometries([pcd])

In [4]:
print(np.asarray(pcd.points))

[[-35.64372253 -21.33800125  -1.08601546]
 [-24.46276093  -9.89589119  -0.52498025]
 [-24.6944809   -9.78285503  -0.52081674]
 ...
 [  4.55649328   5.75053549   0.24058563]
 [  4.59078598   5.89032888   0.24257094]
 [  4.58159828   5.91583157   0.24611443]]


In [7]:
downpcd = pcd.voxel_down_sample(voxel_size=0.5)

In [8]:
o3d.visualization.draw_geometries([downpcd])